### Historical Vessel Positions API


API documentation https://servicedocs.marinetraffic.com/tag/Vessel-Historical-Track#operation/exportvesseltrack

API key can be found at https://www.marinetraffic.com/en/users/my_account/api/account



In [11]:
import io
import csv
import json
import time
import requests
import tempfile
import shutil
import os
from datetime import datetime, timedelta

In [12]:
# Define constants for API call limits and API keys
CALL_LIMIT = 120  # Frequency allowance for API calls in seconds

# Insert your API keys here
API_KEYS = [
    'API_KEY1', 
    'API_KEY2' 
#   'API_KEY3',
#   'API_KEY4',
]

# Editable parameters for the API request
FROMDATE = '2023-01-01'
TODATE = '2023-01-07'
PERIOD = ''  # Options: 'hourly', 'daily', or leave blank
MSGTYPE = ''  # Options: 'simple', 'extended', or leave blank
OUTPUT_DIR = '\\Desktop'  # Default output directory

## Register custom dialect for CSV
csv.register_dialect('custom', delimiter=';', doublequote=True, quotechar='"', lineterminator='\n', escapechar='\\', quoting=csv.QUOTE_MINIMAL)

In [13]:
## Back-end Functions
def build_url(api_key, fromdate, todate, period, msgtype):
    """
    Constructs the URL for the API request based on provided parameters.
    """
    url = f'https://services.marinetraffic.com/api/exportvesseltrack/{api_key}/v:3/fromdate:{fromdate}/todate:{todate}'
    if period:
        url += f'/period:{period}'
    if msgtype:
        url += f'/msgtype:{msgtype}'
    url += '/protocol:jsono'
    return url

def get_user_input(prompt, default_value):
    """
    Prompts the user for input and returns the provided value or a default.
    """
    user_input = input(prompt)
    return user_input if user_input else default_value

def getHISTPositions(url):
    """
    Fetches data from the API and handles potential errors.
    """
    request_time = datetime.now()
    data = []
    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()
    except requests.exceptions.HTTPError as e:
        print(f"\nHTTP error occurred: {e}")
        print(f"Response content: {response.content.decode('utf-8')}")
    except requests.exceptions.RequestException as e:
        print(f"\nAn error occurred: {e}")
    return (request_time, data)

In [1]:
## Functions to build csv
def write_data_to_csv(file, data):
    """
    Writes the API data to a CSV file.
    """
    if not data:
        return
    cols = list(data[0].keys())
    file_writer = csv.DictWriter(file, dialect='custom', fieldnames=cols)
    if file.tell() == 0:
        file_writer.writeheader()
    file_writer.writerows(data)

def save_csv_file(output_dir, file_name, temp_file):
    """
    Saves the temporary CSV file to the specified directory.
    """
    final_path = os.path.join(output_dir, file_name)
    try:
        temp_file.seek(0)
        with open(final_path, 'w', newline='', encoding='utf-8') as f:
            shutil.copyfileobj(temp_file, f)
        temp_file.close()
        print(f"Success: CSV saved at {final_path}")
    except Exception as e:
        print(f"Error: {e}")


## Functions to validate date
def encode_date(date_str):
    """
    Encodes the date string from 'YYYY-MM-DD HH:MM:SS' to 'YYYY-MM-DD%20HH:MM:SS'.
    """
    try:
        # Convert to datetime object and then format for URL encoding
        date_obj = datetime.strptime(date_str, '%Y-%m-%d %H:%M:%S')
        # Format the date and time and replace spaces with '%20'
        encoded_date = date_obj.strftime('%Y-%m-%d %H:%M:%S').replace(' ', '%20')
        return encoded_date
    except ValueError:
        print("Error: Invalid date format. Please use YYYY-MM-DD HH:MM:SS.")
        return None

# Change {MIN_DATE_ALLOWED} based on requirements set
def validate_date_input(date_str):
    """
    Validates that the date input is on or after {MIN_DATE_ALLOWED} and returns the formatted date string.
    """
    try:
        date_obj = datetime.strptime(date_str, '%Y-%m-%d')
        if date_obj < datetime(2022, 1, 1): #Change HERE and Description below
            print("Error: Date must be on or after 2022-01-01.")
            return None
        return date_obj.strftime('%Y-%m-%d')
    except ValueError:
        print("Error: Invalid date format. Please use YYYY-MM-DD.")
        return None

In [ ]:
## Main Function
def main():
    """
    Main function to execute the API data retrieval and file saving process.
    """
    fromdate_input = get_user_input('Enter fromdate (e.g., 2022-01-01): ', FROMDATE)
    todate_input = get_user_input('Enter todate (e.g., 2022-01-07) [MAXIMUM 7 DAYS RANGE]: ', TODATE)
    period = get_user_input('Enter period (e.g., hourly/daily, leave blank if not needed): ', PERIOD)
    msgtype = get_user_input('Enter msgtype (e.g., simple/extended, leave blank if not needed [USE EXTENDED TO GET WEATHER DATA]): ', MSGTYPE)
    output_dir = get_user_input(f'Enter output directory (default is {OUTPUT_DIR}): ', OUTPUT_DIR)

    # Validate and format the date inputs
    fromdate = validate_date_input(fromdate_input)
    todate = validate_date_input(todate_input)
    if fromdate is None or todate is None:
        return

    # If msgtype is 'extended', set period to 'hourly' unless period is already set
    if msgtype == 'extended' and not period:
        period = 'hourly'
        print("Setting period to 'hourly' because msgtype is 'extended' and no period was provided.")

    # Convert input dates to datetime objects
    start_date = datetime.strptime(fromdate, '%Y-%m-%d')
    end_date = datetime.strptime(todate, '%Y-%m-%d')

    # Ensure that todate is within 7 days of fromdate
    max_end_date = start_date + timedelta(days=7)
    if end_date > max_end_date:
        print(f"Error: 'todate' must be within 7 days after 'fromdate'. The maximum allowed 'todate' is {max_end_date.strftime('%Y-%m-%d')}.")
        return

    current_date = start_date

    while current_date <= end_date:
        # Define fromdate and todate for the current day
        fromdate_str = f"{current_date.strftime('%Y-%m-%d')} 00:00:00"
        todate_str = f"{current_date.strftime('%Y-%m-%d')} 23:59:59"

        # Encode dates for URL usage
        fromdate_encoded = encode_date(fromdate_str)
        todate_encoded = encode_date(todate_str)
        if fromdate_encoded is None or todate_encoded is None:
            return

        current_time = datetime.now().strftime('%Y-%m-%d_%H-%M')

        # Generate CSV file name for the current day
        csv_file_name = f'out-api-{current_date.strftime("%Y-%m-%d")}--{current_time}.csv'

        temp_file = tempfile.NamedTemporaryFile(suffix='.csv', prefix='api-historical-', mode='w+', delete=False)

        start_time = datetime.now()

        # Process each API key
        for api_key in API_KEYS:
            current_url = build_url(api_key, fromdate_encoded, todate_encoded, period, msgtype)
            print(f"Request {current_url}")

            req_timestamp, data = getHISTPositions(current_url)
            write_data_to_csv(temp_file, data)
            print(f'Lines: {len(data)}, at {req_timestamp.strftime("%Y-%m-%d %H:%M:%S")}\n')

        # Save the final CSV file for the current day
        save_csv_file(output_dir, csv_file_name, temp_file)

        # Check if waiting time is needed only if there's more than one day to process
        if (end_date - start_date).days > 0 and current_date < end_date:
            elapsed_time = (datetime.now() - start_time).total_seconds()
            if elapsed_time < CALL_LIMIT:
                wait_time = CALL_LIMIT - elapsed_time
                print(f"Waiting for {wait_time} seconds before processing the next day.")
                time.sleep(wait_time)

        # Move to the next day
        current_date += timedelta(days=1)

if __name__ == "__main__":
    main()
